In [1]:
!ls /datasets

 json_to_yolo.py
 materials_seg_ARRAM.zip
 orient.zip
 orient_train.zip
 parts_seg_eyesDcar.zip
'project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1.zip'


In [2]:
NOMBRE_DATASET_USAR = "project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1"
NOMBRE_DATASET_YOLO = "materials_seg_ARRAM"

# funciones auxiliares

In [3]:
import zipfile, os

def descomprimir_zip(archivo_zip, directorio_destino):
    try:
        with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
            zip_ref.extractall(directorio_destino)
        print("¡Archivo ZIP descomprimido correctamente!")
    except zipfile.BadZipFile:
        print("Error: El archivo proporcionado no es un archivo ZIP válido.")
    except Exception as e:
        print(f"Error al descomprimir el archivo ZIP: {e}")

In [4]:
!mkdir /content/CVAT
!mkdir "/content/CVAT/{NOMBRE_DATASET_USAR}"
%cp "/datasets/{NOMBRE_DATASET_USAR}.zip" "/content/CVAT/{NOMBRE_DATASET_USAR}/dataset.zip"
!ls "/content/CVAT/{NOMBRE_DATASET_USAR}"

dataset.zip


In [5]:
# Uso del ejemplo:
archivo_zip = f'/content/CVAT/{NOMBRE_DATASET_USAR}/dataset.zip'  # Ruta y nombre del archivo ZIP que quieres descomprimir
directorio_destino = f'/content/CVAT/{NOMBRE_DATASET_USAR}/'  # Ruta del directorio donde se extraerán los archivos

descomprimir_zip(archivo_zip, directorio_destino)

!ls /content

¡Archivo ZIP descomprimido correctamente!
CVAT


In [6]:
import glob
imagenes_cvat = glob.glob(os.path.join(directorio_destino, "images", "Train", '*.jpg'))
# "Total imagenes: " + str(len([ lab.split(".jpg")[0] for lab in imagenes_cvat ]))

In [7]:
imagenes_cvat_buscar = [img.split(os.sep)[-1] for img in imagenes_cvat]

In [8]:
import xml.etree.ElementTree as ET
import json

# Parsear el archivo XML
archivo_xml = f'/content/CVAT/{NOMBRE_DATASET_USAR}/annotations.xml'

# Cargar y analizar el archivo XML
tree = ET.parse(archivo_xml)
root = tree.getroot()

# Crear el diccionario del dataset
dataset = {}

# Iterar sobre cada elemento 'image'
for image in root.findall('.//image'):
    nombre_imagen = image.get('name')
    if not nombre_imagen in imagenes_cvat_buscar:
        #print("NO: ",nombre_imagen)
        continue
    
    ancho_img = int(image.get('width'))
    alto_img = int(image.get('height'))
    
    # Crear un diccionario para cada imagen
    etiquetas = {'ancho': ancho_img, 'alto': alto_img}

    # Iterar sobre cada polígono dentro de la imagen
    for polygon in image.findall('.//polygon'):
        etiqueta = polygon.get('label')
        coordenadas_str = polygon.get('points')
        # Convertir el string de coordenadas en una lista de listas [x, y]
        coordenadas = [[float(coord) for coord in punto.split(',')] for punto in coordenadas_str.split(';')]

        # Agregar las coordenadas a la lista de la etiqueta correspondiente
        if etiqueta in etiquetas:
            etiquetas[etiqueta].append(coordenadas)
        else:
            etiquetas[etiqueta] = [coordenadas]

    # Agregar el diccionario de etiquetas a la imagen correspondiente
    dataset[nombre_imagen] = etiquetas


#list(dataset.keys())[0], list(dataset.values())[0]
# # Convertir el diccionario a JSON
# json_dataset = json.dumps(dataset, indent=4)

# # Imprimir el JSON resultante
# print(json_dataset)

In [9]:
dataset_filtrado = {imagen: etiquetas for imagen, etiquetas in dataset.items() if len(etiquetas) >= 3}

ejemplo_0 = list(dataset_filtrado.keys())[0]
#ejemplo_0, dataset_filtrado[ejemplo_0]

In [10]:
# Conjunto para almacenar etiquetas únicas
etiquetas_unicas = set()

# Recorrer cada imagen y extraer las etiquetas
for nombre_imagen, etiquetas in dataset.items():
    for etiqueta in etiquetas:
        etiquetas_unicas.add(etiqueta)

# Convertir el conjunto en una lista
lista_etiquetas = sorted(list(etiquetas_unicas))

# Imprimir la lista de etiquetas únicas
#print(lista_etiquetas, len(lista_etiquetas))


# Asignar un ID de clase a cada etiqueta
etiquetas_a_id = {}
for i, etiqueta in enumerate(lista_etiquetas):
    if etiqueta != "alto" and etiqueta != "ancho":
        etiquetas_a_id[etiqueta] = i

print(etiquetas_a_id)



{'Acero corrugado': 0, 'Correa tubular VPT': 1, 'Losa cimentación hormigón': 2, 'Muro hormigón armado 1': 3, 'Muro hormigón prefabricado': 4, 'Pilar rectangular hormigón 50x60': 5, 'Placa alveolar forjado 2': 6, 'Tubería PVC corrugada saneamiento': 7, 'Tubería corrugada polipropileno': 8, 'Viga delta': 9, 'Zapata de hormigón': 10}


In [11]:
!ls "/content/CVAT/{NOMBRE_DATASET_USAR}"

annotations.xml  dataset.zip  images


In [12]:
# list(dataset.items())[0]

In [13]:
import os
import shutil
import json


# Directorio de las imágenes originales
directorio_imagenes = f'/content/CVAT/{NOMBRE_DATASET_USAR}/images/Train'

In [14]:


# Crear estructura de carpetas YOLO
base_dir = f'/content/CVAT/{NOMBRE_DATASET_USAR}/YOLO'
os.makedirs(os.path.join(base_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'labels'), exist_ok=True)

# Procesar el JSON
for nombre_imagen, info in dataset.items():
    ancho_img = info['ancho']
    alto_img = info['alto']

    # Preparar el archivo de etiquetas YOLO
    contenido_etiqueta = []
    for etiqueta, poligonos in info.items():
        if etiqueta in ['ancho', 'alto']:
            continue  # Saltar las claves de ancho y alto
        class_id = etiquetas_a_id.get(etiqueta, -1)
        if class_id == -1:
            continue  # Saltar etiquetas no mapeadas

        for poligono in poligonos:
            # Normalizar y aplanar las coordenadas
            coords = [coord for punto in poligono for coord in punto]
            coords_normalizadas = [str(c / ancho_img if i % 2 == 0 else c / alto_img) for i, c in enumerate(coords)]
            linea = f"{class_id} " + " ".join(coords_normalizadas)
            contenido_etiqueta.append(linea)

    # Escribir el archivo .txt de etiquetas YOLO
    with open(os.path.join(base_dir, 'labels', nombre_imagen.replace('.jpg', '.txt')), 'w') as f:
        f.write('\n'.join(contenido_etiqueta))

    # Mover la imagen
    ruta_origen = os.path.join(directorio_imagenes, nombre_imagen)
    ruta_destino = os.path.join(base_dir, 'images', nombre_imagen)

    try:
        shutil.copy(ruta_origen, ruta_destino)
    except Exception as e:
        print(e)

In [15]:
print("Total imagenes para repartir para YOLO: ", len([ lab.split(".jpg")[0] for lab in glob.glob(os.path.join(directorio_imagenes, '*.jpg')) ]))
#print("Total etiquetas para repartir para YOLO: ", len([ lab.split(".txt")[0] for lab in glob.glob(os.path.join(labels_dir, '*.txt')) ]))

Total imagenes para repartir para YOLO:  288


In [16]:
#nombres_clases = [etiqueta for etiqueta, id in sorted(etiquetas_a_id.items(), key=lambda item: item[1])]
nombres_clases = [etiqueta for etiqueta, id in etiquetas_a_id.items()]
nombres_clases

['Acero corrugado',
 'Correa tubular VPT',
 'Losa cimentación hormigón',
 'Muro hormigón armado 1',
 'Muro hormigón prefabricado',
 'Pilar rectangular hormigón 50x60',
 'Placa alveolar forjado 2',
 'Tubería PVC corrugada saneamiento',
 'Tubería corrugada polipropileno',
 'Viga delta',
 'Zapata de hormigón']

In [17]:
import os
import shutil
import glob
from sklearn.model_selection import train_test_split

# Rutas a las carpetas de imágenes y etiquetas
images_dir = f'/content/CVAT/{NOMBRE_DATASET_USAR}/YOLO/images'
labels_dir = f'/content/CVAT/{NOMBRE_DATASET_USAR}/YOLO/labels'

# Crear la estructura de carpetas para el conjunto de datos
base_dir = f'/content/CVAT/{NOMBRE_DATASET_USAR}/YOLO/dataset'
subsets = ['train', 'test', 'valid']
for subset in subsets:
    os.makedirs(os.path.join(base_dir, subset, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, subset, 'labels'), exist_ok=True)

#!ls "{images_dir}"

In [18]:
images_dir

'/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images'

In [19]:
# Obtener todas las rutas de imágenes
all_images = glob.glob(os.path.join(images_dir, '*.jpg'))
all_images

['/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/DJI_0932_000142.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/DJI_0932_000046.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/Arram2023-12-12-13h15m35s634.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/DJI_0933_000029.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/DJI_0933_000098.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/Arram2023-12-12-14h03m50s966.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/DJI_0932_000145.jpg',
 '/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/images/Arram2023-12-12-14h05m33s954.jpg',
 '/content/CVAT/project_arram materiales-

In [20]:
# Dividir los datos
train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)
test_images, valid_images = train_test_split(test_images, test_size=0.5, random_state=42)

In [21]:
len([ lab.split(".txt")[0] for lab in glob.glob(os.path.join(images_dir, '*.jpg')) ]),len([ lab.split(".txt")[0] for lab in glob.glob(os.path.join(labels_dir, '*.txt')) ])

(288, 288)

In [22]:
len(train_images), len(test_images), len(valid_images)

(230, 29, 29)

In [23]:
#            /content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/labels
#os.listdir("/content/CVAT/project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1/YOLO/labels")

In [24]:
# Función para mover imágenes y etiquetas
def move_files(images, subset):
    for img_path in images:
        folder = os.path.dirname(img_path)
        tmp = folder.split(os.sep)[: -1]
        tmp.append("labels")
        tmp = os.sep.join(tmp)

        name_file = os.path.basename(img_path)

        label_path = os.path.join(tmp, name_file.replace('.jpg', '.txt'))

        # print(label_path, img_path)
        
        #print(os.path.exists(label_path), os.path.exists(img_path))
        #continue

        try:
            shutil.copy(label_path, os.path.join(base_dir, subset, 'labels'))
            shutil.copy(img_path, os.path.join(base_dir, subset, 'images'))
        except Exception as e:
            print(e)

# Mover archivos
move_files(train_images, 'train')
move_files(test_images, 'test')
move_files(valid_images, 'valid')

In [25]:
len([ lab.split(".txt")[0] for lab in glob.glob(os.path.join(images_dir, '*.jpg')) ]),len([ lab.split(".txt")[0] for lab in glob.glob(os.path.join(labels_dir, '*.txt')) ])

(288, 288)

In [26]:


# Crear archivo data.yml
data_yml_content = f"""
train: '{NOMBRE_DATASET_YOLO}/train/images'
val: '{NOMBRE_DATASET_YOLO}/valid/images'
test: '{NOMBRE_DATASET_YOLO}/test/images'

# Classes
nc: {len(nombres_clases)}  # número de clases
names: {nombres_clases}  # lista de nombres de clases
"""

with open(os.path.join(base_dir, 'data.yaml'), 'w') as file:
    file.write(data_yml_content)

In [27]:
import shutil

# Ruta de la carpeta a comprimir
carpeta_a_comprimir = f'/content/CVAT/{NOMBRE_DATASET_USAR}/YOLO/dataset'

# Ruta del archivo zip de salida
ruta_zip = f'/datasets/{NOMBRE_DATASET_YOLO}'

# Crear el archivo zip
shutil.make_archive(ruta_zip, 'zip', carpeta_a_comprimir)

# Devolver la ruta del archivo zip creado
archivo_zip_creado = ruta_zip + '.zip'
print("Archivo comprimido creado en:", archivo_zip_creado)


Archivo comprimido creado en: /datasets/materials_seg_ARRAM.zip


In [28]:
!ls /content/datasets

ls: cannot access '/content/datasets': No such file or directory


In [29]:
!ls /datasets

 json_to_yolo.py
 materials_seg_ARRAM.zip
 orient.zip
 orient_train.zip
 parts_seg_eyesDcar.zip
'project_arram materiales-2023_12_13_15_56_00-cvat for images 1.1.zip'
